In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd

d:\develop\python\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\develop\python\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\develop\python\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\develop\python\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:544: FutureWarning: Passi

In [2]:
data = pd.read_csv('E:\\study\\tensorflow\\data\\Tweets\\Tweets.csv')

In [3]:
data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [4]:
data = data[['airline_sentiment','text']]

In [5]:
data.head()

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...


In [6]:
data.airline_sentiment.unique()

array(['neutral', 'positive', 'negative'], dtype=object)

In [7]:
data.airline_sentiment.value_counts()

negative    9178
neutral     3099
positive    2363
Name: airline_sentiment, dtype: int64

In [8]:
data_n = data[data.airline_sentiment == 'negative']
data_p = data[data.airline_sentiment == 'positive']

In [9]:
data_n = data_n.iloc[:len(data_p)]

In [10]:
len(data_n)

2363

In [11]:
len(data_p)

2363

In [12]:
data = pd.concat([data_n,data_p])

In [13]:
data = data.sample(len(data))

In [14]:
data['review'] = (data.airline_sentiment == 'positive').astype('int')

In [15]:
del data['airline_sentiment']

In [16]:
data

,text,review
3193,@united broke my suitcase &amp; refuses to giv...,0
13855,@AmericanAir Didn't really need anything. Saw ...,1
264,@VirginAmerica thanks for gate checking my bag...,1
1547,"@united Appreciated, but in this case we waite...",0
3162,@united thanks for ruining my wedding annivers...,0
...,...,...
7855,@JetBlue im in a session presented by one of y...,1
3594,"@united Funny thing happened, we arrived on ou...",0
650,@united so you're telling me there is no numbe...,0
2021,"@united still missing my luggage, was promised...",0


In [17]:
import re

In [18]:
token = re.compile('[A-Za-z]+|[!?,.()]')

In [19]:
def reg_text(text):
    new_text = token.findall(text)
    new_text = [word.lower() for word in new_text]
    return new_text

In [20]:
data['text'] = data.text.apply(reg_text)

In [21]:
data

,text,review
3193,"[united, broke, my, suitcase, amp, refuses, to...",0
13855,"[americanair, didn, t, really, need, anything,...",1
264,"[virginamerica, thanks, for, gate, checking, m...",1
1547,"[united, appreciated, ,, but, in, this, case, ...",0
3162,"[united, thanks, for, ruining, my, wedding, an...",0
...,...,...
7855,"[jetblue, im, in, a, session, presented, by, o...",1
3594,"[united, funny, thing, happened, ,, we, arrive...",0
650,"[united, so, you, re, telling, me, there, is, ...",0
2021,"[united, still, missing, my, luggage, ,, was, ...",0


In [22]:
word_set = set()
for text in data['text']:
    for word in text:
        word_set.add(word)

In [23]:
max_word = len(word_set)
max_word

7100

In [24]:
word_list = list(word_set)

In [25]:
word_index = dict((word,word_list.index(word)+1) for word in word_list)

In [26]:
word_index

{'diane': 1,
 'craving': 2,
 'nwk': 3,
 'neworleans': 4,
 'newsbusiness': 5,
 'cbarrows': 6,
 'song': 7,
 'sucked': 8,
 'alavera': 9,
 'calming': 10,
 'black': 11,
 'unexpactable': 12,
 'profit': 13,
 'sibling': 14,
 'you': 15,
 'watching': 16,
 'playlist': 17,
 'much': 18,
 'deal': 19,
 'sun': 20,
 'chn': 21,
 'pairings': 22,
 'individual': 23,
 'cb': 24,
 'washington': 25,
 'frigid': 26,
 'suprlfoi': 27,
 'eq': 28,
 'connolly': 29,
 'mkpognntyc': 30,
 'grandcanyon': 31,
 'mysteriously': 32,
 'daily': 33,
 'showing': 34,
 'believing': 35,
 'tpdkqulm': 36,
 'rescued': 37,
 'let': 38,
 'char': 39,
 'strange': 40,
 'seg': 41,
 'responds': 42,
 'tbsjquw': 43,
 'keepitmovin': 44,
 'suprise': 45,
 'problems': 46,
 'tulsa': 47,
 'hurry': 48,
 'mortified': 49,
 'she': 50,
 'where': 51,
 'sarcastically': 52,
 'sheila': 53,
 'trap': 54,
 'aggravation': 55,
 'kkedjnrtwo': 56,
 'pricey': 57,
 'corny': 58,
 'market': 59,
 'whats': 60,
 'voided': 61,
 'miles': 62,
 'lifeline': 63,
 'rick': 64,
 'di

In [27]:
data_ok = data.text = data.text.apply(lambda x : [word_index.get(word,0) for word in x])

In [28]:
max_len = max(len(x) for x in data_ok)

In [29]:
max_len

40

In [30]:
data_ok = keras.preprocessing.sequence.pad_sequences(data_ok.values,maxlen=max_len)

In [31]:
data_ok.shape

(4726, 40)

In [32]:
data.review.values

array([0, 1, 1, ..., 0, 0, 0])

In [33]:
model = keras.Sequential()

In [34]:
model.add(layers.Embedding(max_word,50,input_length = max_len))

In [35]:
model.add(layers.LSTM(64))

In [36]:
model.add(layers.Dense(1,activation='sigmoid'))

In [37]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 40, 50)            355000    
_________________________________________________________________
lstm (LSTM)                  (None, 64)                29440     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 384,505
Trainable params: 384,505
Non-trainable params: 0
_________________________________________________________________


In [38]:
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['acc'])

In [39]:
model.fit(data_ok,data.review.values,epochs=10,batch_size=128,validation_split=0.2)

Train on 3780 samples, validate on 946 samples
Epoch 1/10
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
 896/3780 [======>.......................] - ETA: 5s - loss: 0.6885 - acc: 0.5279

InvalidArgumentError:  indices[111,38] = 7100 is not in [0, 7100)
	 [[node sequential/embedding/embedding_lookup (defined at d:\develop\python\lib\site-packages\tensorflow_core\python\framework\ops.py:1751) ]] [Op:__inference_distributed_function_3452]

Function call stack:
distributed_function
